#### Task 1

Set up cssandra cluster of three nodes
* https://hub.docker.com/_/cassandra 
* https://gokhanatil.com/2018/02/build-a-cassandra-cluster-on-docker.html 
* https://www.jamescoyle.net/how-to/2448-create-a-simple-cassandra-cluster-with-3-nodes
* https://www.digitalocean.com/community/tutorials/how-to-run-a-multi-node-cluster-database-with-cassandra-on-ubuntu-14-04


Cassandra cluster is configured with docker compose.

Check file *docker-compose.yml* in this repo.

    >docker ps
    CONTAINER ID   IMAGE            COMMAND                  CREATED         STATUS                   PORTS
                            NAMES
    36701a931fa8   cassandra:3.11   "docker-entrypoint.s…"   4 minutes ago   Up 4 minutes (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9044->9042/tcp   node_3
    e67a593bbb42   cassandra:3.11   "docker-entrypoint.s…"   4 minutes ago   Up 4 minutes (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp   node_1
    96fa18536a41   cassandra:3.11   "docker-entrypoint.s…"   4 minutes ago   Up 4 minutes (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9043->9042/tcp   node_2

#### Task 2

Check the cluster configuration with *nodetool status*

    >docker exec node_1 nodetool status
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.29.0.4  323.39 KiB  128          67.0%             9d003c2b-4294-4ddf-856b-9673d0632e3d  west
    UN  172.29.0.3  121.98 KiB  128          63.1%             8419ce92-b05d-4026-8ad2-db9571714255  west
    UN  172.29.0.2  201.89 KiB  128          69.8%             7ebd5415-67dc-44db-a694-e1d0aa099cc0  west

#### Task 3

Using *cqlsh* create three Keyspaces
* https://www.tutorialspoint.com/cassandra/cassandra_create_keyspace.htm
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_keyspace_r.html


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> DESC keyspaces;

    system_traces  system_schema  system_auth  system  system_distributed

    cqlsh> CREATE KEYSPACE music_db_r3 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '3'};
    cqlsh> CREATE KEYSPACE music_db_r2 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '2'};
    cqlsh> CREATE KEYSPACE music_db_r1 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '1'};
    cqlsh> DESC keyspaces;

    system_schema  system       music_db_r2  system_distributed
    system_auth    music_db_r1  music_db_r3  system_traces

    cqlsh> exit

#### Task 4

Create tables in every Keyspace
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_table_r.html
* https://www.tutorialspoint.com/cassandra/cassandra_create_table.htm


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]    
    Use HELP for help.
    cqlsh> DESC keyspaces;

    system_schema  system       music_db_r2  system_distributed
    system_auth    music_db_r1  music_db_r3  system_traces

    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> CREATE TABLE music_by_genre (
                ... genre VARCHAR,
                ... performer VARCHAR,
                ... year INT,
                ... title VARCHAR,
                ... PRIMARY KEY ((genre), performer, year, title)
                ... ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r1> DESC music_by_genre;

    CREATE TABLE music_db_r1.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';

    cqlsh:music_db_r1> USE music_db_r2;
    cqlsh:music_db_r2> CREATE TABLE music_by_genre (
                ... genre VARCHAR,
                ... performer VARCHAR,
                ... year INT,
                ... title VARCHAR,
                ... PRIMARY KEY ((genre), performer, year, title)
                ... ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r2> DESC music_by_genre;

    CREATE TABLE music_db_r2.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';
        
    cqlsh:music_db_r2> USE music_db_r3;
    cqlsh:music_db_r3> CREATE TABLE music_by_genre (
                ... genre VARCHAR,
                ... performer VARCHAR,
                ... year INT,
                ... title VARCHAR,
                ... PRIMARY KEY ((genre), performer, year, title)
                ... ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r3> DESC music_by_genre;

    CREATE TABLE music_db_r3.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';


#### Task 5

Try to read and write to/from different nodes

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1991, 'Smells Like Teen Spirit');
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title
    -------+-----------+------+-------------------------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit

    (1 rows)

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title
    -------+-----------+------+-------------------------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit

    (1 rows)

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title
    -------+-----------+------+-------------------------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit

    (1 rows)

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1963, 'Please Please Me');

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO musics_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1965, 'Help!');
    InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table musics_by_genre"
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1965, 'Help!');
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title
    -------+-------------+------+------------------
    Rock | The Beatles | 1965 |            Help!
    Rock | The Beatles | 1963 | Please Please Me

    (2 rows)

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title
    -------+-------------+------+------------------
    Rock | The Beatles | 1965 |            Help!
    Rock | The Beatles | 1963 | Please Please Me

    (2 rows)

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title
    -------+-------------+------+------------------
    Rock | The Beatles | 1965 |            Help!
    Rock | The Beatles | 1963 | Please Please Me

    (2 rows)

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1975, 'Dark Lady');
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title
    -------+-----------+------+-----------
    Rock | Scorpions | 1975 | Dark Lady

    (1 rows)

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1979, 'Another Piece of Meat');

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title
    -------+-----------+------+-----------------------
    Rock | Scorpions | 1979 | Another Piece of Meat
    Rock | Scorpions | 1975 |             Dark Lady

    (2 rows)

#### Task 6

Insert data to tables and check their distribution between nodes (for each Keyspace use *nodetool status*)
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/insert_r.html
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/select_r.html 
* https://www.tutorialspoint.com/cassandra/cassandra_create_data.htm
* https://www.tutorialspoint.com/cassandra/cassandra_read_data.htm


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1991, 'Drain You');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1989, 'Blew');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1992, 'Mexican Seafood');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1993, 'Pennyroyal Tea');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r1 
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  233.79 KiB  128          33.5%             8419ce92-b05d-4026-8ad2-db9571714255  west
    UN  172.19.0.2  211.47 KiB  128          33.8%             7ebd5415-67dc-44db-a694-e1d0aa099cc0  west
    UN  172.19.0.4  190.45 KiB  128          32.7%             9d003c2b-4294-4ddf-856b-9673d0632e3d  west


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1965, 'Bad Boy');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1969, 'Come Together');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1970, 'Dig It');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1966, 'Yellow Submarine');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r2 
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  225.47 KiB  128          63.1%             8419ce92-b05d-4026-8ad2-db9571714255  west
    UN  172.19.0.2  199.06 KiB  128          69.8%             7ebd5415-67dc-44db-a694-e1d0aa099cc0  west
    UN  172.19.0.4  257.23 KiB  128          67.0%             9d003c2b-4294-4ddf-856b-9673d0632e3d  west


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1990, 'Wind Of Change');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1980, 'The Zoo');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 2010, 'Raised on Rock');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1980, 'Lady Starlight');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r3
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  225.47 KiB  128          100.0%            8419ce92-b05d-4026-8ad2-db9571714255  west
    UN  172.19.0.2  199.06 KiB  128          100.0%            7ebd5415-67dc-44db-a694-e1d0aa099cc0  west
    UN  172.19.0.4  257.23 KiB  128          100.0%            9d003c2b-4294-4ddf-856b-9673d0632e3d  west


#### Task 7

For each record from each keyspace show nodes where they are stored
* https://docs.datastax.com/en/dse/5.1/dse-admin/datastax_enterprise/tools/nodetool/toolsGetEndPoints.html

#### Task 8

Switch of one of the nodes. For each keyspace figure out a consistency level for reading and writing, which of them guranty strong consistency
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/consistency_r.html


#### Task 9

Disable networking leaving all nodes switched on.

#### Task 10

For Keyspace with replication factor 3 make consistency level 1. Write one record (with same primary key but different values to each node.

#### Task 11

Turn on networking and figure out whicj value is desided to be agreed by cluster.

#### Task 12

Check the behavour of lightweight transactions for previous tasks in partitioned and consolidated cluster.
* https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertLWT.html 
